###### Data Engineering Capstone Project

# US Student Immigration
> The purpose of this project is to study the foreign students. The goal is to offer Data teams Analysts a selection of data concerning immigration to the United States.

#### Project Summary

The project follows the follow steps:
* [Step 1: Scope the Project and Gather Data](#Step-1:-Scope-the-Project-and-Gather-Data)

* [Step 2: Explore and Assess the Data](#Step-2:-Explore-Assess-the-Data) 

* [Step 3: Define the Data Model](#Step-3:-Define-the-Data-Model)
* [Step 4: Run ETL to Model the Data](#Step-4:-Run-ETL-to-Model-the-Data)
* [Step 5: Complete Project Write Up](#Step-5:-Complete-Project-Write-Up)

In [1]:
import os
import io
import re
import sys
import datetime
import pandas as pd
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import *
import pyspark.sql.functions as func
import datetime as dt
pd.set_option('display.max_colwidth', 200)
pd.set_option("display.precision", 2)

In [2]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.hadoop:hadoop-aws:2.7.3 pyspark-shell"

# Step 1: Scope the Project and Gather Data

Data warehouse allow us to collect, transform and manage data from varied sources. Then, Data Team Business connect to it and analyse data. 
Apache Spark has been used to gather data
Amazon S3 buckets store the data in parquet files for the Data teams.
The main dataset includes data on immigration to the United State.
The questions about foreign students and their choice to come to US may be useful to propose services.   
How many students arrived in US in April?    
Which Airline bring the most student in April?    
What are the top city to arrive in the USA?   
Where are from?   
what are the student profils (age, country born, country indicators)? 

#### Data Source

[Datactionnary](2_data_dictionnary.ipynb) provides informations about dataset and tables used. [This notebook](1_Exploration_python.ipynb) performs a first exploration with Python and explain the datasets, which variables I kept. 

Dataset |File |Data Source|Dataframe Name
-|-|-|-|
I94 Immigration | immigration_data_sample.csv| [US National Tourism and Trade Office](https://travel.trade.gov/research/programs/i94/description.asp)| df_immigration
I94 Description Labels  Description|I94_SAS_Labels_Descriptions.SAS |US National Tourism and Trade Office|
Global Land Temperature|GlobalLandTemperaturesByCity.csv| [Berkeley Earth](http://berkeleyearth.org/)|df_temperature
Global Airports|airports-extended.csv| [OpenFlights.org and user contributions](https://www.kaggle.com/open-flights/airports-train-stations-and-ferry-terminals)|df_global_airports
Airports codes |airport-codes_csv.csv| provide by Udacity|df_airport_code
Iso country | wikipedia-iso-country-codes.csv|[Wikipedia](https://gist.github.com/radcliff/f09c0f88344a7fcef373)|df_iso_country
US Cities Demographic| us-cities-demographics.csv|provide by Udacity|df_demograph
Indicators developpment| WDIData.csv| [World Bank](https://www.kaggle.com/xavier14/wdidata)|df_indicator_dev
Education-statistics| EdStatsData.csv|provide by Kaggle [World Bank](https://www.kaggle.com/kostya23/worldbankedstatsunarchived)|df_Educ_data

##### I94 Immigration Data
* Source: https://travel.trade.gov/research/reports/historical/2016.html
    * data 'data/18-83510-I94-Data-2016', provide one file per month
        * These records are described according to 28 variables and 3M  rows per file
        *  It's provide information about Arrival/Departure to foreign visitors        
    * I94_SAS_Labels_Description.SAS for variable descriptions
    
##### Global Land Temperature Data
* Source: http://berkeleyearth.org/
    * data 'GlobalLandTemperaturesByCity.csv' provide climate information
        * Each line correspond to a record of temperature per day from city around the world.
        * The GlobalLandTemperaturesByCity.csv has 7 variables and 8599213 rows.
        
##### Global Airports Data
* Source: https://www.kaggle.com/open-flights/airports-train-stations-and-ferry-terminals
    * data 'airports-extended.csv'. Some of the data come from public sources and some of it comes from OpenFlights.org user contributions.
        * It's provide informatioms about of airports, train stations, and ferry terminals around the world.
        * There are 4 variables in 'airports-extended.csv'and 10668 rows
        
##### Airports Data Description Data
* Source: https://datahub.io/core/airport-codes#data
    * airport-codes_csv.csv. The airport code refers to the IATA airport code, 3 letters code unique for all airports in the world
        * The airport-codes_csv.csv provides informations about aiports.
        * There are 55075 rows and 12 columns in airport-codes_csv.csv.
        
##### Iso country Data
* Source: https://gist.github.com/radcliff/f09c0f88344a7fcef373
    * data 'wikipedia-iso-country-codes.csv'. This is a database about the different code useful to identify country.
        * This table gives us informations about Country codes used to identify each country
        * There are 4 variables and 247 rows.
        
##### US cities Demographics Data
* Source: https://data.census.gov/cedsci/. 
    * data 'us-cities-demographics.csv'. This dataset contains information about the demographics of all US cities and come from the US Census Bureau.
        * Provides simple informations about US State population
        * Contains 12 variables and 2892 rows
        
##### World Development Indicators Data
* Source: https://www.kaggle.com/xavier14/wdidata
    * data 'WDIData.csv'. The primary World Bank collection of development indicators, compiled from officially-recognized international sources. 
        * It presents the most current and accurate global development data available, and includes national, regional and global estimates.
        * Contains 64 variables, most of which are variables per year(1960 to 2018), with economics context and 422137 rows.
        
##### Education statistics Data
* Source: https://www.kaggle.com/kostya23/worldbankedstatsunarchived
    * data 'education-statistics/EdStatsData.csv' 
        * This dataset contains 64 variables witheducation context , most of which are variables per year(1970 to 2100) and 886931 rows. 
        
##### i94addr Data
* Source: I94_SAS_Labels_Description.SAS
    * US States code defined in I94_SAS_Labels_Description.SAS
        * data 'i94addr.csv' provides State Id and State name  
        
##### i94city_i94res Data
* Source: I94_SAS_Labels_Description.SAS
    * data 'i94cit_i94res.csv' defined Code Country by 3 digits
        * data 'i94cit_i94res.csv' provides Country Id and Country name
        
##### i94mode Data
* Source: I94_SAS_Labels_Description.SAS
    * data 'i94mode.csv' defined arrival US
        * data 'i94mode.csv' provides code Mode and name Code.
        
##### i94port Data
* Source: I94_SAS_Labels_Description.SAS
    * data 'i94port.csv'
        * data 'i94port.csv' provides Port Id, Port city and State Id.
        
##### i94visa Data
* Source: I94_SAS_Labels_Description.SAS
    * data 'i94visa.csv'
        * data 'i94visa.csv' povides code Visa ans Visa

### SETUP SPARK AND ENVIRONMENT

In [3]:
output_parquet = '../../output/'
path = '../../data/'
date_time = datetime.today().strftime('%Y%m%d')

In [4]:
def create_spark_session():
    spark = SparkSession.builder \
                    .appName("Us_student_immigation") \
                    .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11") \
                    .enableHiveSupport() \
                    .getOrCreate()
    return spark
spark = create_spark_session()

%autosave 60

Autosaving every 60 seconds


In [5]:
#spark.sparkContext.getConf().getAll()

# enhaut

# [DOWN](#enbas)

### I94 Immigration Data
#### Exploration

* Path = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
* There are 3096313 rows and 29 columns in *i94_apr16_sub.sas7bdat*.
* Name of the dataFrame: df_immigration
* As we see in [data exploration file](./0_dataset_information.ipynb), some variables are either not present or not very present (visapost, occup, entdepu, insnum)
* Variables droped: depdate, count, occup, entdepa, entdepd, entdepu, matflag, biryear, insnum, dtadfile, visapost, fltno, admnum, insnum.  	
* Variables used:

Column Name | Description |
-|-|
**cicid**|     ID uniq per record in the dataset 
**i94yr**|     4 digit year  
**i94mon**|    Numeric month 
**i94cit**|     3 digit code of source city for immigration (Born country) 
**i94res**|    3 digit code of source country for immigration
**i94port**|   Port addmitted through 
**arrdate**|   Arrival date in the USA
**i94mode**|   Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported) 
**i94addr**|   State of arrival 
**i94bir**|    Age in years 
**i94visa**|   Visa Code - 1 = Business / 2 = Pleasure / 3 = Student
**gender**|    Gender
**dtaddto**|   Date to which admitted to U.S. (allowed to stay until)
**airline**|   Airline used to arrive in U.S.
**admnum**|    Admission number, should be unique and not nullable 
**visatype**|  Class of admission legally admitting the non-immigrant to temporarily stay in U.S.

#### Read I94 data

In [6]:
def load_immigration(path, file):
    df = spark.read \
        .format('com.github.saurfang.sas.spark') \
        .option('header', 'true') \
        .load(path+file)
    nb_rows = df.count()
    print(f'*****         Loading {nb_rows} rows')
    print(f'*****         Display the Schema')
    df.printSchema()
    print(f'*****         Display few rows')
    df.show(3, truncate = False)
    return df, nb_rows

In [7]:
file = '18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'

In [8]:
file = '18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
# refaire avec S3 et tous les fichiers (get_path_sas_folder parquet file)
df_immigration, rows_immig = load_immigration(path, file)

*****         Loading 3096313 rows
*****         Display the Schema
root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-

#### Write to parquet files

In [9]:
pattern = re.search(r'((i94_[a-z]{3}[1-9]{2}))', file).group(0)
df = df_immigration

name_file = pattern + '_staging.parquet_' + date_time
print(f' Path to file Parquet is:   "{output_parquet}{name_file}"')
df.write.mode("overwrite").parquet(f'{output_parquet}{name_file}')
print(f' Done for "{name_file}" !')

path_i94_immigration = output_parquet + name_file

 Path to file Parquet is:   "../../output/i94_apr16_staging.parquet_20200526"
 Done for "i94_apr16_staging.parquet_20200526" !


### Global Land Temperature Data
#### Exploration
* Path = '../../data/GlobalLandTemperaturesByCity.csv
* There are 8599212 rows and 7 columns in *GlobalLandTemperaturesByCity.csv*.
* Name of the dataFrame: df_temperature

* As we see in [data exploration file](./0_dataset_information.ipynb), the first date is in 1743, and we find a row per day per town. So we will make aggregation for this data set and drop 'AverageTemperature' , 'Latitude' and 'Longitude' columns
* Variables used:

Column Name | Description 
-|-|
**dt**|Date format YYYY-MM-DD| 
**AverageTemperature**|Average Temperature for the city to th date dt|
**City**| City name| 
**Country**| Country name |


#### Read GlobalLandTemperaturesByCity

In [10]:
def load_temperature(path, file):
    df = spark.read \
        .format("csv") \
        .option('header', 'true') \
        .option('inferSchema', 'true') \
        .load(path+file)
    nb_rows = df.count()
    print(f'*****         Loading {nb_rows} rows')
    print(f'*****         Display the Schema')
    df.printSchema()
    print(f'*****         Display few rows')
    df.show(3, truncate = False)
    return df, nb_rows

In [11]:
file = 'GlobalLandTemperaturesByCity.csv'
df_temperature, rows_temp = load_temperature(path, file)

*****         Loading 8599212 rows
*****         Display the Schema
root
 |-- dt: timestamp (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)

*****         Display few rows
+-------------------+------------------+-----------------------------+-----+-------+--------+---------+
|dt                 |AverageTemperature|AverageTemperatureUncertainty|City |Country|Latitude|Longitude|
+-------------------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01 00:00:00|6.068             |1.7369999999999999           |Århus|Denmark|57.05N  |10.33E   |
|1743-12-01 00:00:00|null              |null                         |Århus|Denmark|57.05N  |10.33E   |
|1744-01-01 00:00:00|null              |null                         |Å

#### Write to parquet files

In [12]:
pattern = re.search(r'(.*)\.csv$', file).group(1)
df = df_temperature

name_file = pattern + '_staging.parquet_' + date_time
print(f' Path to file Parquet is:   "{output_parquet}{name_file}"')
df.write.mode("overwrite").parquet(f'{output_parquet}{name_file}')
print(f' Done for "{name_file}" !')

path_temperature = output_parquet + name_file

 Path to file Parquet is:   "../../output/GlobalLandTemperaturesByCity_staging.parquet_20200526"
 Done for "GlobalLandTemperaturesByCity_staging.parquet_20200526" !


### Airports Code Data
#### Exploration
* Path = '../../data/airport-codes_csv.csv'
* There are 55075 rows and 12 column in *airport-codes_csv.csv*
* Name of the DataFrame : df_airport_code
* Some variables left more 50% of data (continent, iata_code and local_code) so I kept:

Column Name | Description 
-|-|
**ident**| Unique identifier Airport code|
**type**| Type of airport | 
**name**| Name of the airport | 
**continent**| Continent | | 
**iso_country**| ISO code of airport country |
**iso_region**| ISO code of the region airport | 
**municipality**| City name where the airport is located | 
**iata_code**| IATA code of the airport|

#### Read Airports Code

In [13]:
def load_airport_code(path, file):
    df = spark.read \
        .format("csv") \
        .option('header', 'true') \
        .option('inferSchema', 'true') \
        .load(path+file)
    nb_rows = df.count()
    print(f'*****         Loading {nb_rows} rows')
    print(f'*****         Display the Schema')
    df.printSchema()
    print(f'*****         Display few rows')
    df.show(3, truncate = False)
    return df, nb_rows

In [14]:
file = 'airport-codes_csv.csv'    
df_airport_code, rows_code = load_airport_code(path, file)

*****         Loading 55075 rows
*****         Display the Schema
root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

*****         Display few rows
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+----------------------------------+
|ident|type         |name                |elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|coordinates                       |
+-----+-------------+--------------------+------------+---------+--------

#### Write to parquet files

In [15]:
pattern = re.search(r'(.*)\.csv$', file).group(1)
df = df_airport_code

name_file = pattern + '_staging.parquet_' + date_time
print(f' Path to file Parquet is:   "{output_parquet}{name_file}"')
df.write.mode("overwrite").parquet(f'{output_parquet}{name_file}')
print(f' Done for "{name_file}" !!!')

path_aiport_code = output_parquet + name_file

 Path to file Parquet is:   "../../output/airport-codes_csv_staging.parquet_20200526"
 Done for "airport-codes_csv_staging.parquet_20200526" !!!


### Global Airports Data
#### Exploration
* Path = '../../data/airports-extended.csv'
* There are 10668 rows and 13 columns in *airports-extended.csv*
* Name of the dataframe : df_global_airports
* No missing value, and I kept:

Column Name | Description | Example | Type
-|-|-|-|
**airport_name**|Name of airport|Nadzab Airport|Object
**airport_city**|Main city served by airport|Nadzab|Object
**airport_country**|Country or territory where airport is located|Papua New Guinea|Object
**airport_iata**|3-letter IATA code|LAE|Object

#### Read airports-extended

In [16]:
global_airports_schema = T.StructType([
    T.StructField('airport_ID', T.IntegerType(), False),
    T.StructField('name', T.StringType(), False),
    T.StructField('city', T.StringType(), False),
    T.StructField('country', T.StringType(), False),
    T.StructField('iata', T.StringType(), False),
    T.StructField('icao', T.StringType(), False),
    T.StructField('latitude', T.StringType(), False),
    T.StructField('longitude', T.StringType(), False),
    T.StructField('altitude', T.IntegerType(), False),
    T.StructField('timezone', T.StringType(), False),
    T.StructField('dst', T.StringType(), False),
    T.StructField('tz_timezone', T.StringType(), False),
    T.StructField('type', T.StringType(), False),
    T.StructField('data_source', T.StringType(), False)
])

In [17]:
def load_global_airports(path, file):
    df = spark.read \
        .format("csv") \
        .option('header', 'True') \
        .option('inferSchema', 'true') \
        .schema(global_airports_schema) \
        .load(path+file)
    nb_rows = df.count()
    print(f'*****         Loading {nb_rows} rows')
    print(f'*****         Display the Schema')
    df.printSchema()
    print(f'*****         Display few rows')
    df.show(3, truncate = False)
    return df, nb_rows

In [18]:
file = 'airports-extended.csv'
df_global_airports, rows_global = load_global_airports(path, file)

*****         Loading 10667 rows
*****         Display the Schema
root
 |-- airport_ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- altitude: integer (nullable = true)
 |-- timezone: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- tz_timezone: string (nullable = true)
 |-- type: string (nullable = true)
 |-- data_source: string (nullable = true)

*****         Display few rows
+----------+----------------------------+-----------+----------------+----+----+------------------+------------------+--------+--------+---+--------------------+-------+-----------+
|airport_ID|name                        |city       |country         |iata|icao|latitude          |longitude         |altitude|timezone|dst|tz_timezone         |type   |d

#### Write to parquet files

In [19]:
pattern = re.search(r'(.*)\.csv$', file).group(1)
df = df_global_airports

name_file = pattern + '_staging.parquet_' + date_time
print(f' Path to file Parquet is:   "{output_parquet}{name_file}"')
df.write.mode("overwrite").parquet(f'{output_parquet}{name_file}')
print(f' Done for "{name_file}" !!!')

path_global_airports = output_parquet + name_file

 Path to file Parquet is:   "../../output/airports-extended_staging.parquet_20200526"
 Done for "airports-extended_staging.parquet_20200526" !!!


### Iso Country Data
#### Exploration
* Path = '../../data/wikipedia-iso-country-codes.csv
* There are 246 rows and 5 columns in *wikipedia-iso-country-codes.csv*
* Name of the dataframe: df_iso_country
* I remove 'ISO 3166-2' column, only one missing value. I choose to replace manually. 

Column Name | Description 
-|-|
**Country_name**|Country Name in English|
**Alpha2_code**|code 2 letter code for the country|
**Alpha3_code**|code 3 letter code for the country|
**Numeric_code**|ISO 3166-2 code|

#### Read wikipedia-iso-country-codes

In [20]:
iso_country_schema = T.StructType([
    T.StructField('Country', T.StringType(), False),
    T.StructField('Alpha_2', T.StringType(), False),
    T.StructField('Alpha_3', T.StringType(), False),
    T.StructField('Num_code', T.StringType(), False),
    T.StructField('ISO_3166-2', T.StringType(), True),    
]) 

In [21]:
def load_iso_country(path, file):
    df = spark.read \
        .format("csv") \
        .option('header', 'true') \
        .option('inferSchema', 'true') \
        .schema(iso_country_schema) \
        .load(path+file)
    nb_rows = df.count()
    print(f'*****         Loading {nb_rows} rows')
    print(f'*****         Display the Schema')
    df.printSchema()
    print(f'*****         Display few rows')
    df.show(3, truncate = False)
    return df, nb_rows

In [22]:
file = 'wikipedia-iso-country-codes.csv'
df_iso_country, rows_iso = load_iso_country(path, file)

*****         Loading 246 rows
*****         Display the Schema
root
 |-- Country: string (nullable = true)
 |-- Alpha_2: string (nullable = true)
 |-- Alpha_3: string (nullable = true)
 |-- Num_code: string (nullable = true)
 |-- ISO_3166-2: string (nullable = true)

*****         Display few rows
+--------+-------+-------+--------+-------------+
|Country |Alpha_2|Alpha_3|Num_code|ISO_3166-2   |
+--------+-------+-------+--------+-------------+
|Zimbabwe|ZW     |ZWE    |716     |ISO 3166-2:ZW|
|Zambia  |ZM     |ZMB    |894     |ISO 3166-2:ZM|
|Yemen   |YE     |YEM    |887     |ISO 3166-2:YE|
+--------+-------+-------+--------+-------------+
only showing top 3 rows



#### Write to parquet files

In [23]:
pattern = re.search(r'(.*)\.csv$', file).group(1)
df = df_iso_country

name_file = pattern + '_staging.parquet_' + date_time
print(f' Path to file Parquet is:   "{output_parquet}{name_file}"')
df.write.mode("overwrite").parquet(f'{output_parquet}{name_file}')
print(f' Done for "{name_file}" !!!')

path_iso_country = output_parquet + name_file

 Path to file Parquet is:   "../../output/wikipedia-iso-country-codes_staging.parquet_20200526"
 Done for "wikipedia-iso-country-codes_staging.parquet_20200526" !!!


### US cities Demographics
#### Exploration
* Path = '../../data/us-cities-demographics.csv
* There are 2891 rows and 12 columns in us-cities-demographics.csv
* Dataframe name : df_demograph
* Missing less than 1% in some variables so I drop 'Number of Veterans', 'Average Household Size' and kept: 

Column Name | Description | 
-|-|
**City**|Name of the city|
**State**|US state of the city|
**Median Age**|The median of the age of the population|
**Male Population**|Number of the male population|
**Female Population**|Number of the female population|
**Total Population**|Number of the total population|
**Foreign-born**|Number of residents of the city that were not born in the city|
**State Code**|Code of the state of the city|
**Race**|Race class|
**Count**|Number of individual of each race|

#### Read us-cities-demographics

In [24]:
demograph_schema = T.StructType([
    T.StructField('City', T.StringType(), False),
    T.StructField('State', T.StringType(), False),
    T.StructField('Median_Age', T.FloatType(), False),
    T.StructField('Male_Population', T.IntegerType(), False),
    T.StructField('Female_Population', T.IntegerType(), False),
    T.StructField('Total_Population', T.IntegerType(), False),
    T.StructField('Number_of_Veterans', T.IntegerType(), False),
    T.StructField('Foreign-born', T.IntegerType(), False),
    T.StructField('Average_Household_Size', T.FloatType(), False),
    T.StructField('State_Code', T.StringType(), False),
    T.StructField('Race', T.StringType(), False),
    T.StructField('Count', T.IntegerType(), False)
]) 

In [25]:
def load_demograph(path, file):
    df = spark.read \
        .format("csv") \
        .option('header', 'true') \
        .option('delimiter', ';') \
        .option('inferSchema', 'true') \
        .schema(demograph_schema) \
        .load(path+file)
    nb_rows = df.count()
    print(f'*****         Loading {nb_rows} rows')
    print(f'*****         Display the Schema')
    df.printSchema()
    print(f'*****         Display few rows')
    df.show(3, truncate = False)
    return df, nb_rows

In [26]:
file = 'us-cities-demographics.csv'
df_demograph, rows_demo = load_demograph(path, file)

*****         Loading 2891 rows
*****         Display the Schema
root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median_Age: float (nullable = true)
 |-- Male_Population: integer (nullable = true)
 |-- Female_Population: integer (nullable = true)
 |-- Total_Population: integer (nullable = true)
 |-- Number_of_Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average_Household_Size: float (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)

*****         Display few rows
+-------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+------------------+-----+
|City         |State        |Median_Age|Male_Population|Female_Population|Total_Population|Number_of_Veterans|Foreign-born|Average_Household_Size|State_Code|Race              |Count|
+-----

#### Write to parquet files

In [27]:
pattern = re.search(r'(.*)\.csv$', file).group(1)
df = df_demograph

name_file = pattern + '_staging.parquet_' + date_time
print(f' Path to file Parquet is:   "{output_parquet}{name_file}"')
df.write.mode("overwrite").parquet(f'{output_parquet}{name_file}')
print(f' Done for "{name_file}" !!!')

path_demograph = output_parquet + name_file

 Path to file Parquet is:   "../../output/us-cities-demographics_staging.parquet_20200526"
 Done for "us-cities-demographics_staging.parquet_20200526" !!!


### World Development Indicators Data
#### Exploration 
* Path = '../../data/WDIData.csv
* There are 422136 rows and 64 columns in *WDIData.csv*
* Dataframe name : df_indicator_dev
* This dataset contains 64 variables with economics context , most of which are variables per year(1960 to 2018). Data is missing a lot, between 40% and 91%. I just need the year 2015 to explain the Economic context in the country and make aggregation per country. I kept:

Column Name | Description | 
-|-|
**Country Name**|Name of the country|
**Country Code**|3 letters code of country|
**Indicator Name**|indicators of economic development|conversion factor, GDP (LCU per inter...|
**Indicator Code**|letters indicator code|
**2016**|one column per year since 1960|

#### Read WDIData

In [28]:
def load_indicator_dev(path, file):
    df = spark.read \
        .format("csv") \
        .option('header', 'true') \
        .option('inferSchema', 'true') \
        .load(path+file) \
        .select("Country Name","Country Code", "Indicator Name", "Indicator Code", "2015" ) \
        .toDF("Country_Name","Country_Code", "Indicator_Name", "Indicator_Code", "2015")
    nb_rows = df.count()
    print(f'*****         Loading {nb_rows} rows')
    print(f'*****         Display the Schema')
    df.printSchema()
    print(f'*****         Display few rows')
    df.show(3, truncate = False)
    return df, nb_rows


In [29]:
file = 'WDIData.csv'
df_indicator_dev, rows_dev = load_indicator_dev(path, file)

*****         Loading 422136 rows
*****         Display the Schema
root
 |-- Country_Name: string (nullable = true)
 |-- Country_Code: string (nullable = true)
 |-- Indicator_Name: string (nullable = true)
 |-- Indicator_Code: string (nullable = true)
 |-- 2015: double (nullable = true)

*****         Display few rows
+------------+------------+-------------------------------------------------------------------------+-----------------+----------------+
|Country_Name|Country_Code|Indicator_Name                                                           |Indicator_Code   |2015            |
+------------+------------+-------------------------------------------------------------------------+-----------------+----------------+
|Arab World  |ARB         |2005 PPP conversion factor, GDP (LCU per international $)                |PA.NUS.PPP.05    |null            |
|Arab World  |ARB         |2005 PPP conversion factor, private consumption (LCU per international $)|PA.NUS.PRVT.PP.05|null         

#### Write to parquet files

In [30]:
pattern = re.search(r'(.*)\.csv$', file).group(1)
df = df_indicator_dev

name_file = pattern + '_staging.parquet_' + date_time
print(f' Path to file Parquet is:   "{output_parquet}{name_file}"')
df.write.mode("overwrite").parquet(f'{output_parquet}{name_file}')
print(f' Done for "{name_file}" !!!')

path_indicator_dev = output_parquet + name_file

 Path to file Parquet is:   "../../output/WDIData_staging.parquet_20200526"
 Done for "WDIData_staging.parquet_20200526" !!!


### Education statistics Data
#### Exploration
* Path = 'education-statistics/EdStatsData.csv'
* There are 886930 rows and 64 columns in *EdStatsData.csv*
* DtaFrane name : 'df_Educ_data'
* There more missing value with *EdStatsData.csv* than *WDIData.csv*. I just need the year 2015 to explain the Education context in the country and make aggregation per country. I kept:
 
Column Name | Description 
-|-|
**Country Name**|Name of the country|
**Country Code**|3 letters code of country|
**Indicator Name**|indicators of education development|
**Indicator Code**|letters indicator code|
**1970 ...2100**|one column per year since 1970|

#### Read EdStatsData

In [31]:
def load_Educ_data(path, file):
    df = spark.read \
        .format("csv") \
        .option('header', 'true') \
        .option('inferSchema', 'true') \
        .load(path+file) \
        .select("Country Name","Country Code", "Indicator Name", "Indicator Code", "2015" ) \
        .toDF("Country_Name","Country_Code", "Indicator_Name", "Indicator_Code", "2015")
    nb_rows = df.count()
    print(f'*****         Loading {nb_rows} rows')
    print(f'*****         Display the Schema')
    df.printSchema()
    print(f'*****         Display few rows')
    df.show(3, truncate = False)
    return df, nb_rows


In [32]:
file = 'education-statistics/EdStatsData.csv'
df_educ_data, rows_educ = load_Educ_data(path, file)

*****         Loading 886930 rows
*****         Display the Schema
root
 |-- Country_Name: string (nullable = true)
 |-- Country_Code: string (nullable = true)
 |-- Indicator_Name: string (nullable = true)
 |-- Indicator_Code: string (nullable = true)
 |-- 2015: double (nullable = true)

*****         Display few rows
+------------+------------+-----------------------------------------------------------------------+--------------+----+
|Country_Name|Country_Code|Indicator_Name                                                         |Indicator_Code|2015|
+------------+------------+-----------------------------------------------------------------------+--------------+----+
|Arab World  |ARB         |Adjusted net enrolment rate, lower secondary, both sexes (%)           |UIS.NERA.2    |null|
|Arab World  |ARB         |Adjusted net enrolment rate, lower secondary, female (%)               |UIS.NERA.2.F  |null|
|Arab World  |ARB         |Adjusted net enrolment rate, lower secondary, gender 

#### Write to parquet files

In [33]:
pattern = re.search(r'(.*)\.csv$', file).group(1)
df = df_educ_data

name_file = pattern + '_staging.parquet_' + date_time
print(f' Path to file Parquet is:   "{output_parquet}{name_file}"')
df.write.mode("overwrite").parquet(f'{output_parquet}{name_file}')
print(f' Done for "{name_file}" !!!')

path_educ_data = output_parquet + name_file

 Path to file Parquet is:   "../../output/education-statistics/EdStatsData_staging.parquet_20200526"
 Done for "education-statistics/EdStatsData_staging.parquet_20200526" !!!


In [34]:
### Create Parquet Files 
# Parse I94_SAS_Labels_Description.SAS and save in parquet format in '../../data/'
file = 'I94_SAS_Labels_Descriptions.SAS'
!python parse_file.py $path $file
#### Read Parquet files create from 'I94_SAS_Labels_Descriptions.SAS'

Running "../../data/I94_SAS_Labels_Descriptions.SAS"
 
There are 583 rows in i94port.parquet
There are 3 rows in i94visa.parquet
There are 55 rows in i94addr.parquet
There are 289 rows in i94cit_i94res.parquet
There are 4 rows in i94mode.parquet
 
***** Make i94 labels files is done!


### I94 Description Labels  Description

Here some data extract from '../../data/I94_SAS_Labels_Description.SAS'.    
To explain code in I94-immigration, I create 5 files and read here. The file was cleaned and parsed with the scrip *parse_file.py*. see below:

In [35]:
path = '../../data/'
file = 'I94_SAS_Labels_Descriptions.SAS'

In [36]:
print(path+file)

../../data/I94_SAS_Labels_Descriptions.SAS


In [37]:
%pycat parse_file.py

In [38]:
%run -i parse_file.py $path $file

Running "../../data/I94_SAS_Labels_Descriptions.SAS"
 
There are 583 rows in i94port.parquet
There are 3 rows in i94visa.parquet
There are 55 rows in i94addr.parquet
There are 289 rows in i94cit_i94res.parquet
There are 4 rows in i94mode.parquet
 
***** Make i94 labels files is done!


----

# Step 2: Explore Assess the Data

#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [34]:
!ls -la ../../output_parquet

0_dataset_information.ipynb  2_exploration_spark.ipynb	 metastore_db
1_Exploration_python.ipynb   capstone_exploration.ipynb  parse_file.py
2_data_dictionnary.ipynb     derby.log			 postgresql-42.2.10.jar


In [39]:
# Read Parquet files
path = '../data/'
df_immigration = spark.read.parquet(path+path_i94_immigration)
df_temperature = spark.read.parquet(path+path_temperature)
df_airport_code = spark.read.parquet(path+path_aiport_code)
df_global_airports = spark.read.parquet(path+path_global_airports)
df_iso_country = spark.read.parquet(path+path_iso_country)
df_demograph = spark.read.parquet(path+path_demograph)
df_indicator_dev = spark.read.parquet(path+path_indicator_dev)
df_educ_data = spark.read.parquet(path+path_educ_data)

In [33]:
print(path+path_i94_immigration)

../../data/../../output/i94_apr16_staging.parquet_20200526


In [40]:
display(f'df_airport_code: There are {df_airport_code.count()} rows from parquet file, {rows_code} before staging')
display(f'df_demograph : There are {df_demograph.count()} rows from parquet file, {rows_demo} before staging')
display(f'df_educ_data : There are {df_educ_data.count()} rows from parquet file, {rows_educ} before staging')
display(f'df_global_airports : There are {df_global_airports.count()} rows from parquet file, {rows_global} before staging')
display(f'df_immigration : There are {df_immigration.count()} rows from parquet file, {rows_immig} before staging')
display(f'df_indicator_dev : There are {df_indicator_dev.count()} rows from parquet file, {rows_dev} before staging')
display(f'df_iso_country : There are {df_iso_country.count()} rows from parquet file, {rows_iso} before staging')
display(f'df_temperature : There are {df_temperature.count()} rows from parquet file, {rows_temp} before staging')

'df_airport_code: There are 55075 rows from parquet file, 55075 before staging'

'df_demograph : There are 2891 rows from parquet file, 2891 before staging'

'df_educ_data : There are 886930 rows from parquet file, 886930 before staging'

'df_global_airports : There are 10667 rows from parquet file, 10667 before staging'

'df_immigration : There are 3096313 rows from parquet file, 3096313 before staging'

'df_indicator_dev : There are 422136 rows from parquet file, 422136 before staging'

'df_iso_country : There are 246 rows from parquet file, 246 before staging'

'df_temperature : There are 8599212 rows from parquet file, 8599212 before staging'

In [41]:
i94_mode = pd.read_parquet(output_parquet+'i94mode.parquet')
print(f'***** Dataframe i94_mode *****')
print("There are {} rows.".format(len(i94_mode)))
print(' ')

i94_ctry = pd.read_parquet(output_parquet+'i94cit_i94res.parquet')
print(f'***** Dataframe i94_ctry *****')
print("There are {} rows.".format(len(i94_ctry)))
print(' ')

i94_addr = pd.read_parquet(output_parquet+'i94addr.parquet')
print(f'***** Dataframe i94_addr *****')
print("There are {} rows.".format(len(i94_addr)))
print(' ')

i94_visa = pd.read_parquet(output_parquet+'i94visa.parquet')
print(f'***** Dataframe i94_visa *****')
print("There are {} rows.".format(len(i94_visa)))
print(' ')

i94_port = pd.read_parquet(output_parquet+'i94port.parquet')
print(f'***** Dataframe i94_port *****')
print("There are {} rows.".format(len(i94_port)))
print(' ')

***** Dataframe i94_mode *****
There are 4 rows.
 
***** Dataframe i94_ctry *****
There are 289 rows.
 
***** Dataframe i94_addr *****
There are 55 rows.
 
***** Dataframe i94_visa *****
There are 3 rows.
 
***** Dataframe i94_port *****
There are 583 rows.
 


In [27]:
print(output_parquet)
print(path)

../../output/
../../data/


In [42]:
%who_ls DataFrame


['df',
 'df_airport_code',
 'df_demograph',
 'df_educ_data',
 'df_global_airports',
 'df_immigration',
 'df_indicator_dev',
 'df_iso_country',
 'df_temperature',
 'i94_addr',
 'i94_ctry',
 'i94_mode',
 'i94_port',
 'i94_visa']

### I94 Immigration Data
* i94addr, missing 152592 values (code State US, 2 letters)
    * fill by Port_id from the dataframe 'i94port' 
    * join on 'df_immigration.i94port == port_state_dic.Port_id', with no missing values
    * nul value replace by State_id
* int_col = ['cicid', 'i94yr', 'i94mon','i94cit', 'i94res', 'i94mode', 'i94bir', 'i94visa']
    * fill null by default value from dictionnary and cast the int_col in Integer
* str_cols = ['i94addr', 'i94port', 'gender', 'airline', 'visatype']
    * fill null by default value from dictionnary
* date_col = ['arrdate'(double sas format),'dtadfile'(string YYYYMMDD), 'dtaddto'(string DDMMYYYY)]
    * 'arrdate' in SAS date format, a value represents the number of days between January 1, 1960, and a other date.
    * cast the date and fill the null value

In [32]:
print(path_i94_immigration)

../../output/i94_apr16_staging.parquet_20200526


In [30]:
immigration = spark.read.parquet(path+path_i94_immigration)


AnalysisException: 'Path does not exist: file:/home/output/i94_apr16_staging.parquet_20200526;'

In [210]:
#drop columns with a lot of null values and not useful
drop_col = ['depdate', 'count', 'occup', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'biryear', \
            'insnum','visapost', 'fltno', 'admnum', 'insnum']
immigration = immigration.drop(*drop_col)

In [212]:
immigration.createOrReplaceTempView("temp_immig")
#Count of Missing values of dataframe in pyspark 
df_immigration.select([count(when(isnan(c), c)).alias(c) for c in df_immigration.columns]).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,i94bir,i94visa,dtadfile,dtaddto,gender,airline,visatype
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [225]:
#Count of null values of dataframe in pyspark 
immigration.select([count(when(col(c).isNull(), c)).alias(c) for c in immigration.columns]).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,i94bir,i94visa,dtadfile,dtaddto,gender,airline,visatype
0,0,0,0,0,0,0,0,239,152592,802,0,1,477,414269,83627,0


In [226]:
# get number of null value in df_immigration.i94port
immigration.filter(immigration.i94port.rlike('[A-Z]{3}')) \
              .filter(immigration.i94addr.isNull()) \
              .select(immigration.i94port, df_immigration.i94addr).count()

152079

In [229]:
# Clean df_immigration.i94addr by df_immigration.i94port == port_state_dic.Port_id
# port_state_dic = [{'Port_id': 'alc', 'State_id': 'ak'}]
# Result: clean 148379 rows

# create dictionnary from i94_port
port_state_dic = dict([(i,a) for i, a in zip(i94_port.Port_id,i94_port.State_id)])
# lamda function to get State_id

user_func =  udf(lambda x: port_state_dic.get(x))
# Take col i94addr when isNull(), replace by state_id by lambda otherwise keep null
newdf = df_immigration.withColumn('i94addr', F.when((F.col('i94addr').isNull()),
                                       user_func(df_immigration.i94port)).otherwise(F.col('i94addr')))
# get number of null value in df_immigration.i94port
newdf.filter(newdf.i94port.rlike('[A-Z]{3}')) \
     .filter(newdf.i94addr.isNull()) \
     .select(newdf.i94port, newdf.i94addr)


DataFrame[i94port: string, i94addr: string]

In [230]:
# replace the null value and cast the columns in integer
# int_col = ['cicid', 'i94yr', 'i94mon','i94cit', 'i94res', 'i94mode', 'i94bir', 'i94visa']
null_int = {'cicid': -1, 'i94yr': -1, 'i94mon': -1,'i94cit': 239, 'i94res': 239, 'i94mode': 9, 'i94bir': -1, 'i94visa': -1}
for k in null_int:
        newdf = newdf.withColumn(k, F.when((F.col(k).isNull()), null_int[k])
                 .otherwise(F.col(k).cast("int")))
        
# replace the null value for the string
# str_cols = ['i94addr', 'i94port', 'gender', 'airline', 'visatype']
null_str = {'i94addr': '99', 'i94port': '999', 'gender': 'U', 'airline': 'unknown', 'visatype': '99' }
for k in null_str:
        newdf = newdf.withColumn(k, F.when((F.col(k).isNull()), null_str[k])
                                 .otherwise(F.col(k)))
        
# date_col = ['arrdate'(double sas format),
#             'dtadfile'(string YYYYMMDD), 
#             'dtaddto'(string DDMMYYYY)]
null_date = {'arrdate': 'NA', 'dtadfile': 'NA', 'dtaddto': 'NA' }
setup_date = udf(lambda x: (datetime(1960, 1, 1). date() + dt.timedelta(x)).isoformat() if x else None)
newdf = newdf.withColumn("arrdate", setup_date(newdf.arrdate)) \
    .withColumn("dtadfile",to_date(unix_timestamp(col("dtadfile"),"yyyyMMdd").cast("timestamp"))) \
    .withColumn("dtaddto",to_date(unix_timestamp(col("dtaddto"),"MMddyyyy").cast("timestamp")))
for k in null_date:
        newdf = newdf.withColumn(k, F.when((F.col(k).isNull()), null_date[k])
                                 .otherwise(F.col(k)))
        
newdf = newdf.withColumn('dtadfile', to_date(newdf.dtadfile, 'yyyyMMdd'))
newdf = newdf.withColumn('dtaddto', to_date(newdf.dtadfile, 'yyyyMMdd'))
newdf = newdf.withColumn('arrdate', to_date(newdf.dtadfile, 'yyyyMMdd'))

In [231]:
newdf.select([count(when(col(c).isNull(), c)).alias(c) for c in newdf.columns]).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,i94bir,i94visa,dtadfile,dtaddto,gender,airline,visatype
0,0,0,0,0,0,0,3096313,0,0,0,0,3096313,3096313,0,0,0


In [219]:
newdf= newdf.dropDuplicates()

In [223]:
newdf = (newdf.withColumnRenamed("cicid", "id_i94") \
             .withColumnRenamed("i94yr", "year") \
             .withColumnRenamed("i94mon", "month") \
             .withColumnRenamed("i94cit", "country_born_iso") \
             .withColumnRenamed("i94res", "country_res_iso") \
             .withColumnRenamed("i94port", "iata_code") \
             .withColumnRenamed("arrdate", "arr_date") \
             .withColumnRenamed("i94mode", "arri_mode") \
             .withColumnRenamed("i94addr", "state_id_arrival") \
             .withColumnRenamed("i94bir", "age") \
             .withColumnRenamed("i94visa", "arr_reason") \
             .withColumnRenamed("dtadfile", "dt_add_i94") \
             .withColumnRenamed("dtaddto", "depar_max") \
             .withColumnRenamed("gender", "gender") \
             .withColumnRenamed("airline","airline") \
             .withColumnRenamed("visatype:", "visatype"))


In [224]:
newdf.printSchema()

root
 |-- id_i94: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- country_born_iso: integer (nullable = true)
 |-- country_res_iso: integer (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- arr_date: date (nullable = true)
 |-- arri_mode: integer (nullable = true)
 |-- state_id_arrival: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- arr_reason: integer (nullable = true)
 |-- dt_add_i94: date (nullable = true)
 |-- depar_max: date (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- visatype: string (nullable = true)



In [ ]:
df_immigration = newdf.collect()

### Global Land Temperature Data



* Path = '../../data/GlobalLandTemperaturesByCity.csv
* There are 8599212 rows and 7 columns in *GlobalLandTemperaturesByCity.csv*.
* Name of the dataFrame: df_temperature

* As we see in [data exploration file](./0_dataset_information.ipynb), the first date is in 1743, and we find a row per day per town. So we will make aggregation for this data set and drop 'AverageTemperature' , 'Latitude' and 'Longitude' columns
* Variables used:

Column Name | Description 
-|-|
**dt**|Date format YYYY-MM-DD| 
**AverageTemperature**|Average Temperature for the city to th date dt|
**City**| City name| 
**Country**| Country name |



* i94addr, missing 152592 values (code State US, 2 letters)
    * fill by Port_id from the dataframe 'i94port' 
    * join on 'df_immigration.i94port == port_state_dic.Port_id', with no missing values
    * nul value replace by State_id
* int_col = ['cicid', 'i94yr', 'i94mon','i94cit', 'i94res', 'i94mode', 'i94bir', 'i94visa']
    * fill null by default value from dictionnary and cast the int_col in Integer
* str_cols = ['i94addr', 'i94port', 'gender', 'airline', 'visatype']
    * fill null by default value from dictionnary
* date_col = ['arrdate'(double sas format),'dtadfile'(string YYYYMMDD), 'dtaddto'(string DDMMYYYY)]
    * 'arrdate' in SAS date format, a value represents the number of days between January 1, 1960, and a other date.
    * cast the date and fill the null value

In [ ]:
df_immigration.count()

In [ ]:
df_temperature = spark.read.parquet(path+path_temperature)

### enbas

### [UP](#enhaut)

In [ ]:

def get_key(my): 
    for key, value in port_state_dic.items():
        if my == key: 
            return(key, value)
        if my == value:
                return(key, value)
print(get_key('XT'))


In [ ]:
# i94_mode
i94_mode = pd.read_parquet(path+'i94mode.parquet')
#i94_mode.head(5)
# dictionnary
mode_dic = i94_mode.set_index("Mode_id")["Mode"].to_dict()

#display(i94_mode.head())
#i94_mode = pd.read_csv(path+'i94mode.csv')
#display(mode_dic)
#{'1': 'air', '2': 'sea', '3': 'land', '9': 'not reported'}

# i94cit_i94res
path = '../../data/'
i94_ctry = pd.read_parquet(path+'i94cit_i94res.parquet')
#i94_ctry.head(5)
# dictionnary
ctry_dic = i94_ctry.set_index("Country_id")['Country'].to_dict()

#display(ctry_dic)
#{582: 'mexico',
# 236: 'afghanistan',
# 101: 'albania',...}
#display(i94_ctry.head())

# i94_addr
path = '../../data/'
i94_addr = pd.read_parquet(path+'i94addr.parquet')
i94_addr.head(5)
# dictionnary
addr_dic = i94_addr.set_index("State_id")['State'].to_dict()
#display(addr_dic)
#{'al': 'alabama',
# 'ak': 'alaska',
# 'az': 'arizona',...}
#display(i94_addr.head())

# i94_visa
path = '../../data/'
i94_visa = pd.read_parquet(path+'i94visa.parquet')
#i94_visa.head(5)
# dictionnary
visa_dic = i94_visa.set_index('Code_visa')['Visa'].to_dict()

#display(visa_dic)
# {1: 'business', 2: 'pleasure', 3: 'student'}
#display(i94_visa.head())

# i94_port
path = '../../data/'
i94_port = pd.read_parquet(path+'i94port.parquet')
#i94_port.head(5)
# dictionnary
port_dic= dict([(i,[a,b]) for i, a,b in zip(i94_port.Port_id, i94_port.Port_city,i94_port.State_id)])
port_state_dic = dict([(i,a) for i, a in zip(i94_port.Port_id,i94_port.State_id)])

# each row becomes a dictionary where key is column name and value is the data in the cell
# [{'Port_id': 'alc', 'Port_city': 'alcan', 'State_id': 'ak'},
# {'Port_id': 'anc', 'Port_city': 'anchorage', 'State_id': 'ak'},...]
#port_dic = i94_port.to_dict('records')
#display(port_dic)
#display(i94_port.head())

#display(port_state_dic)
#display(port_dic)
#{'alc': ['alcan', 'ak'],
# 'anc': ['anchorage', 'ak'],
# 'bar': ['baker aaf - baker island', 'ak'],

In [ ]:
# if not duplicates, the twice values are the same
display(f"There are {df_immigration.distinct().count()} distinc values")
display(f"There are {df_immigration.count()} values") 

In [ ]:
df.printSchema()

In [ ]:
#Count of null values of dataframe in pyspark 
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas()

In [ ]:
#drop_columns = ['cicid', 'i94yr', 'i94mon', 'i94visa', 'gender', 'airline', 'visatype']

In [ ]:
newdf.where(col("i94addr").isNull()).filter(newdf.i94port =='MIA').show()


In [ ]:
#Count of null values of dataframe in pyspark 
newdf.select([count(when(col(c).isNull(), c)).alias(c) for c in newdf.columns]).toPandas()

In [ ]:
valuesA = [('',1, 'lun'),('Monkey',2, 'mar'),('',3, 'mer'),('Darth Vader',4, 'jeu')]
TableA = spark.createDataFrame(valuesA,['name','myid', 'day'])
# Put null in empty cell
TableA = TableA.withColumn('name', when(col('name') == '', None).otherwise(col('name')))


valuesB = [('Rutabaga',1),('Pirate',2),('Ninja',3),('Darth Vader',4)]
TableB = spark.createDataFrame(valuesB,['name','myid'])
 
TableA.show()
TableB.show()
TableA.createOrReplaceTempView("TableA")
TableB.createOrReplaceTempView("TableB")

TableA.alias('A').join(TableB.alias('B'), on='myid', how='left') \
                 .select(
                        'myid',
                        'day',
                        F.when(
                            F.isnull(F.col('A.name')),
                            F.col('B.name')
                        ).otherwise(F.col('A.name')).alias('name')
                    ) \
                 .show()

In [ ]:
Basically below line of code check all 5 SAL fields and if it is null, replace it with 0. 
If not keep the original value.

df1 = df.withColumn("SAL1", when(df.SAL1.isNull(), lit(0)).otherwise(df.SAL1))\
.withColumn("SAL2", when(df.SAL2.isNull(), lit(0)).otherwise(df.SAL2))\
.withColumn("SAL3", when(df.SAL3.isNull(), lit(0)).otherwise(df.SAL3))\
.withColumn("SAL4", when(df.SAL4.isNull(), lit(0)).otherwise(df.SAL4))\
.withColumn("SAL5", when(df.SAL5.isNull(), lit(0)).otherwise(df.SAL5))\





In [ ]:
ta.join(tb, ta.myid == tb.myid, 'left').select(tb.myid, coalesce(ta.name, tb.name)).show(truncate=False)

In [ ]:
int_col = ['cicid', 'i94yr', 'i94mon','i94cit', 'i94res', 'i94mode', 'i94bir', 'i94visa']


In [ ]:
------